In [21]:
import sqlite3
import string
from random import choice,randint

# Create logindata

In [22]:
charlist = string.ascii_letters

In [23]:
pwlist = string.punctuation + string.ascii_letters + '0123456789'

In [24]:
def uname(p):
    return uname(p+1) + choice(charlist) if p < 3 else choice(charlist)

In [25]:
def pw(checklist,p,l):
    if p < l:
        char = choice(pwlist)
        if char in string.punctuation: checklist[0] = True 
        elif char in string.ascii_letters: checklist[1] = True
        else: checklist[2] = True
        return pw(checklist,p+1,l) + char
    else:
        if not all(checklist):
            if not checklist[0]: 
                checklist[0] = True
                return pw(checklist,p-1,l) + choice(string.punctuation)
            if not checklist[1]:
                checklist[1] = True
                return pw(checklist,p-1,l) + choice(string.ascii_letters)
            if not checklist[2]:
                checklist[2] = True
                return pw(checklist,p-1,l) + choice('0123456789') 
        return choice(pwlist)

### Write datas to sql db

In [26]:
con = sqlite3.connect('bookdbase.db')
cursor = con.cursor()
for i in range(1,104):
    cursor.execute('insert into logindata values (?,?,?)',(i,uname(0),pw([False,False,False],0,randint(8,16))))
    con.commit()
con.close()

# CREATE RECORDS_ID TABLES

In [27]:
con = sqlite3.connect('bookdbase.db')
cursor = con.cursor()
for i in range(1,104):
    cursor.execute(f'CREATE TABLE records_{i} (id INT,Name TEXT, Kind TEXT, Author TEXT,IsFiction INT, Page INT, Rating FLOAT,Note TEXT)')
    con.commit()
con.close()

# CREATE BOOKS FOR EACH USER

### resource for booklist:  <a>https://www.theguardian.com/books/2019/sep/21/best-books-of-the-21st-century</a>

In [28]:
import requests
from bs4 import BeautifulSoup

response = requests.get("https://www.theguardian.com/books/2019/sep/21/best-books-of-the-21st-century")
soup = BeautifulSoup(response.content, 'html.parser')
book2author = {}

datas = [elm.text for elm in soup.find_all('span',{'class':'dcr-d3izhq'}) if elm.text not in [str(a) for a in range(101)]+['48 ']]

for i in range(len(datas)):
    if i % 2 == 0:
        try:
            book2author[datas[i]] = datas[i + 1].split(' ', 3)[1:3]
        except:
            print('dict created!')

names = [name[:34] if len(name) > 34 else name for name in book2author.keys()]
authors = [' '.join(author)[:17] for author in book2author.values()]
kindlist =  ['adventure', 'classics', 'crime', 'fairy tales', 'fables',
                             'fantasy', 'historical fict.', 'horror', 'humour and satire', 'literary fiction',
                             'mystery', 'poetry', 'plays', 'romance', 'science fiction', 'short stories',
                             'thrillers', 'war', "women’s fiction",
                        'autobiography', 'biography', 'essays', 'history', 'politics',
                        'non-fiction novel', 'self-help']
kinds = [choice(kindlist) for i in range(100)]


def get_fiction(kind):
    return 1 if kind in ['adventure', 'classics', 'crime', 'fairy tales', 'fables',
                             'fantasy', 'historical fict.', 'horror', 'humour and satire', 'literary fiction',
                             'mystery', 'poetry', 'plays', 'romance', 'science fiction', 'short stories',
                             'thrillers', 'war', "women’s fiction"] else 0
is_fiction = [get_fiction(kinds[i]) for i in range(100)]

pages = [randint(0,1499) for i in range(100)]


namekind = {}
for i in range(len(names)):
    namekind[names[i]] = choice(kinds)
kinds = [namekind[name] for name in names]
con = sqlite3.connect('bookdbase.db')
cursor = con.cursor()
for u in range(3,104):
    for i in range(randint(30,100)):
        cursor.execute(f'insert into records_{u} VALUES(?,?,?,?,?,?,?,?)',(u,names[i],kinds[i],
                                                                       authors[i],get_fiction(kinds[i]),
                                                                           pages[i],randint(0,100)/10,'No notes added!'))
        con.commit()
con.close()